In [3]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit
from sklearn.model_selection import train_test_split
%matplotlib inline

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier   
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

import csv

In [4]:
training_values = pd.read_csv("sample_data/assignment_2019/training_values.csv")
training_labels = pd.read_csv("sample_data/assignment_2019/training_labels.csv")
cities = pd.read_csv("sample_data/assignment_2019/cities.csv")
training_values.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011/03/14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013/03/06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013/02/25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013/01/28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011/07/13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [20]:
for i in training_labels.columns:
    print(training_labels.loc[2,i])
# len(training_values.columns)

34310
functional


In [10]:
machine = training_values.loc[:,['construction_year','quantity']]

machine['function'] = training_labels.loc[:,'status_group']

for feature in machine.columns:
    if feature != 'construction_year':
        a = list(machine.loc[:,feature].unique())
        for n, m in enumerate(a):
            machine = machine.replace({m: n})

X_train, X_test, y_train, y_test = train_test_split(machine.loc[:,:'quantity'],
                                                    machine.loc[:,'function'],            
                                                    random_state=0)

clf=RandomForestClassifier(n_estimators=250)
clf.fit(X_train,y_train)
print("acc :", clf.score(X_test,y_test))

0 enough
1 insufficient
2 dry
3 seasonal
4 unknown
0 functional
1 non functional
2 functional needs repair
acc : 0.661952861952862


In [5]:
predict_values = pd.read_csv("sample_data/assignment_2019/test_values.csv")

X_predict = predict_values.loc[:,['construction_year','quantity']]

for feature in X_predict.columns:
    if feature != 'construction_year':
        a = list(X_predict.loc[:,feature].unique())
        for n, m in enumerate(a):
            X_predict = X_predict.replace({m: n})

file =  predict_values[['id']]
file['status_group'] = clf.predict(X_predict)
file['status_group'] = file['status_group'].replace({0:'functional',1:'non functional',2:'functional needs repair'})

a = file.to_numpy()
fnameCSV = 'SubmissionFormat.csv'

with open(fnameCSV, 'w') as fid:    
    writer = csv.writer(fid)
    writer.writerow("id,status_group")
    for aa in a:        
        writer.writerow(aa)

/Users/boomchawit/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/boomchawit/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
